In [1]:
import tensorflow as tf
import os
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def rankBasedDropout(layer, batchSize, batchXS, batchYS,PH_X,PH_Y):

    #maxValueAtEachBatch = tf.Variable(tf.random_normal([None, lenOfZ_layer],stddev=0.01))

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

    # maxValueAtEachBatch(tensor)를 기본 리스트형으로 변환.
    maxValueAtEachBatch = tf.reduce_max(layer, 1)
    maxValueAtEachBatch_toScalar = maxValueAtEachBatch.eval(session=sess,
                                                            feed_dict={PH_X: batchXS, PH_Y:batchYS, keep_prob:0.8})

    w,h = 2,batchSize
    TwoDimArray_BatchIdx_MaxValue = [[0 for x in range(w)] for y in range(h)]

    for i in range(batchSize):
        TwoDimArray_BatchIdx_MaxValue[i][0] = i
        TwoDimArray_BatchIdx_MaxValue[i][1] = maxValueAtEachBatch_toScalar[i]

    TwoDimArray_BatchIdx_MaxValue.sort(key=lambda x:x[1])

    #layer(tensor)를 기본 리스트형으로 변환.
    layer_toScalar = layer.eval(session=sess,
                                feed_dict={PH_X: batchXS, PH_Y: batchYS})

    for i in range(batchSize):
        k = TwoDimArray_BatchIdx_MaxValue[i][0]
        layer_toScalar[k] *= (i/batchSize)

    #layer(기본형)을 tensor로 변환.
    layer = tf.convert_to_tensor(layer_toScalar,dtype=tf.float32)
    sess.close()

    return layer



In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

mnist = input_data.read_data_sets("./mnist/data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data\t10k-images-idx3-ubyte.gz
Extracting ./mnist/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

batchSize = 3

batchXS, batchYS = mnist.train.next_batch(batchSize)

W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
model1 = tf.matmul(X,W1)
L1 = tf.nn.relu(model1)
L1 = rankBasedDropout(L1, batchSize, batchXS, batchYS,X,Y)
L1 = tf.layers.batch_normalization(L1, training= True)

W2 = tf.Variable(tf.random_normal([256,256], stddev=0.01))
model2 = tf.matmul(L1, W2)
L2 = tf.nn.relu(model2)
L2 = rankBasedDropout(L2, batchSize,batchXS,batchYS,X,Y)
L2 =tf.layers.batch_normalization(L2, training= True)

W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model3 = tf.matmul(L2, W3)


In [5]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model3, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


total_batch = int(mnist.train.num_examples/batchSize)

